<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gradient Without Backpropagation

In [57]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm

In [12]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 18.5 MB/s eta 0:00:00


In [13]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the dataset

In [14]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



In [16]:
# Alternative way to get the first element: image, label = next(iter(train_data))
for image, label in train_data:
  break

In [17]:
# Simple transformation that converts the PIL image to a PyTorch array
T.ToTensor()(image).shape

torch.Size([1, 28, 28])

In [20]:
# T.Compose allows to chain together multiple transformations
train_transforms = T.Compose([
    T.ToTensor()
])

In [21]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=train_transforms)

In [22]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [23]:
xb, yb = next(iter(train_loader))
print(xb.shape)
print(yb.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [24]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Initialize parameters ????!!!!!

In [25]:
#Initialize parameters

def init():
  theta = torch.randn(X.shape, requires_grad=False)
  v = torch.randn(X.shape, requires_grad=False)
  return theta, v

Define Convolutional Neural Network Class

In [26]:
from torch import nn
from torch.nn import functional as F

In [44]:
class SimpleCNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        input_size = 1
        self.conv1 = nn.Conv2d(input_size, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        self.max_pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.max_pool(x)
        x = x.reshape((-1, 64*7*7))
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [45]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [46]:
# We need to move the SimpleCNN model to the device immediately
cnn = SimpleCNN(1).to(device)

In [47]:
# Note: we also need to move data when asking for a prediction
cnn(xb.to(device)).shape

torch.Size([64, 10])

Train and evaluate the network with forward gradient

In [54]:
def accuracy(net, loader, device):
  # A function that aggregates the accuracy over all mini-batches in the loader.
  # See here for a quick-start on torchmetrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html.
  #acc = torchmetrics.Accuracy().to(device)
  acc = torchmetrics.Accuracy('multiclass', num_classes=10).to(device)
  for xb, yb in loader:
      xb, yb = xb.to(device), yb.to(device)
      ypred = cnn(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [55]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(cnn, test_loader, device)

tensor(0.0661)

In [56]:
# Note: it is important to move the CNN to the device before initializing the optimizer,
# since the optimizer also has a state that must be moved to the GPU.
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(cnn.parameters())

In [ ]:
for epoch in range(10):
  
  cnn.train()
  for xb, yb in tqdm.tqdm(train_loader):
    
    xb, yb = xb.to(device), yb.to(device)
    
    

  cnn.eval()
  print(f'Accuracy at epoch {epoch}: {accuracy(cnn, test_loader, device)}')